# Query and visualize your graph

Cypher is a declarative query language for graphs, and should be easy to get started with for anyone who is familiar with SQL. Kuzu’s implementation of Cypher is based on the [openCypher standard](https://opencypher.org/).

## Query interfaces

There are three ways you can query an existing Kuzu database:
- **CLI**: Write Cypher queries directly in the Kuzu CLI shell.
- **Kuzu Explorer**: Interactively write and run Cypher queries using [Kuzu Explorer](https://kuzudb.github.io/visualization/kuzu-explorer/), a web-based query interface that also provides graph visualization.
- **Client APIs**: Integrate Kuzu in your application using your preferred programming language.

## Cypher introduction

For simplicity, we’ll demonstrate how to run Cypher queries using Kuzu Explorer. You are welcome to run the same queries via the CLI or your preferred client language APIs.

The advantage of using Kuzu Explorer to run queries during the prototyping and exploration phase is that you can visualize the graph as you query it. Additionally, you can view the query results as a table or as JSON, allowing you to easily design and test your application logic.

### Open a database in Kuzu Explorer

Run the appropriate version of Kuzu Explorer via Docker as follows:

```bash
docker run --rm -p 8000:8000 \
    -v /path/to/example.kuzu:/database \
    kuzudb/explorer:latest

docker run -p 8000:8000 \
           -v /path/to/local/directory:/database \
           -e KUZU_FILE=example.kuzu \
           --rm kuzudb/explorer:latest
```

And access via `http://localhost:8000/`

**Note**

You can alternatively install a specific version of Kuzu Explorer by replacing latest with the version number, such as `kuzudb/explorer:0.xx.0`. The explorer version must match the Kuzu version used to create any existing database in the provided path to ensure storage format compatibility.

In [1]:
%%bash

docker run -d \
    -p 8000:8000 \
    -v /home/locch/Works/graph-exp/08_kuzu-quickstart:/database \
    kuzudb/explorer:latest

96ca57024e3502c2a47e58e12dad5b99d202eb79c7b571bf221ec2c06d5f9c46


## Visualize nodes and relationships

Our first Cypher query will use the `MATCH` statement to display all nodes and relationships from all tables:

```cypher
MATCH (a)-[b]->(c)
RETURN *;
```
Note that this is a sample graph with only a small number of nodes and relationships. For real-world graphs, you will likely want to limit the number of returned nodes and relationships by using the LIMIT clause.

```cypher
MATCH (a)-[b]->(c)
RETURN *
LIMIT 10;
```

## Match nodes by label

You can match on a particular node label using its table name as shown below:

```cypher
MATCH (a:User)
RETURN a.name AS name, a.age AS age;
```

## Match relationships by label

```cypher
MATCH (a)-[e:Follows]->(b)
RETURN a.name AS user1, b.name AS user2, e.since AS follows_since;
```

## Equality predicates on node/relationship properties

```cypher
MATCH (a:User)-[e:Follows {since: 2020}]->(b:User {name: "Karissa"})
RETURN b.name AS user, a.name AS follower, e.since AS follows_since;
```

# Run prepared Cypher statements

Similar to [SQL injection](https://en.wikipedia.org/wiki/SQL_injection) in relational databases, it is possible to have Cypher injection in graph databases. This is when a malicious input can manipulate dynamically generated Cypher queries to execute arbitrary code. To avoid such security issues, it is generally a good practice to provide parameters to your Cypher queries instead of using concatenated string queries.

## Why use parameters?

Consider a scenario where you want to retrieve all persons in a database who are older than a certain age. Instead of recreating the Cypher query with a hardcoded age value, it is more efficient to pass the age as a parameter to the query, so that the same query can be reused for different values.

## ✅ Recommended

The `$min_age` and `$max_age` variables in the Cypher query are mapped to the `min_age` and `max_age` keys in the `parameters` dictionary.

```python
min_age = 18
max_age = 30

response = conn.execute(
    """
    MATCH (p:Person)
    WHERE p.age > $min_age AND p.age < $max_age
    RETURN p.name;
    """,
    parameters={"min_age": min_age, "max_age": max_age}
)

response
```

## ❌ Not recommended

Although it is possible to pass the min_age and max_age variables directly into the query as string literals, this is strongly discouraged.

```python
min_age = 18
max_age = 30

conn.execute(
    f"""
    MATCH (p:Person)
    WHERE p.age > {min_age} AND p.age < {max_age}
    RETURN p.name;
    """
)
```

# Scan data from various sources

Cypher supports the `LOAD FROM` clause to scan data from various sources. Scanning is the act of reading data from a source, but not inserting it into the database. Inserting data into the database involves “copying”; see the [Import data](https://kuzudb.github.io/import) section for details.

Scanning data using `LOAD FROM` is useful to inspect a subset of your data, understand its structure, and perform transformations like rearranging columns.

## General usage

Say you have a `user.csv` file that looks like this:

```csv
name,age
Adam,30
Karissa,40
Zhang,50
Noura,25
```

You can scan the file and count the number of rows:

```cypher
LOAD FROM "user.csv" (header = true)
RETURN COUNT(*);
```

You can also apply filter predicates via the `WHERE` clause, like this:

```cypher
LOAD FROM "user.csv" (header = true)
WHERE age > 25
RETURN COUNT(*);
```

Note that when scanning from CSV files, Kuzu will attempt to auto-cast the data to the correct type when possible. For example, the `age` column is cast to an `INT64` type.

You can reorder the columns by simply returning them in the order you want. The `LIMIT` keyword can be used to limit the number of rows returned. The example below returns the first two rows, with the `age` and `name` columns in the order specified.

```cypher
LOAD FROM "user.csv" (header = true)
RETURN age, name
LIMIT 2;
```

## More features

`LOAD FROM` is a general-purpose clause in Cypher for scanning data from various data sources, including files and in-memory DataFrames. See the detailed documentation below for more details on how to use the `LOAD FROM` clause.